# Naive Bayes

- Spam or no Spam

> The code below can be found in the book as well with minor changes....

**The reason this technique is considered `naive` (or simplistic) is that it makes some simple yet extreme assumptions.**  
- In probability terms, we say the events are mutually exclusive.

**What are the events?**  
- The occurrence of one word in the `email` (or any text) is assumed to have no connection with another word.

Nowadays, this assumption can be outright rejected, but the point is: it's a simple yet powerful technique that utilises one of the key concepts of probability to make wise predictions.

The formula representing the above assumption is:

S - event: "message is spam".
&
contain the words like `bitcoin` & `rolex`..

$$
P(X_1 = x_1, \ldots, X_n = x_n \mid S) = P(X_1 = x_1 \mid S) \times \cdots \times P(X_n = x_n \mid S)
$$

> This model was quite popularly used as a Spam filter.

Using the same Bayes’ theorem logic as in the `bitcoin-only` spam filter, we can calculate the probability that a message is spam with this formula:

$$
P(S \mid X = x) = \frac{P(X = x \mid S)}{P(X = x \mid S) + P(X = x \mid \neg S)}
$$

-------
Before we jump on to create our `spam filter`:

Few more things to take care of:
1. As there can be a lot of words in an email, to calculate probability for every vocabulary word and then multiplying them together can result into **underflow** problem.
   > - Floating point numbers too close to 0.
   > - Easier way to deal with them is to use `log`
2. Calculate estimate for $P(Xi|S)$ and $P(Xi|¬S)$
   > - probabilities that a spam message / non-spam message contains the word $w_i$.
   > - To estimate probabilities like $P(X_i|S)$ and $P(X_i|\neg S)$, we use the fraction of spam / nonspam messages containing the word $w_i$.
However, if a word appears only in nonspam messages, we might estimate $P(w_i|S) = 0$, which causes issues for classification.



To fix this, we apply **smoothing** using a pseudocount $k$. 

The smoothed estimate becomes:

$$P(X_i|S) =\frac{k + ws}{2k+ ms}$$

- $ws$ = number of spams containing word $w_i$
- $ms$ = number of spam messages

This prevents zero probabilities and ensures the classifier can handle rare words.

Similarly for $P(X_i | \neg S)$

------------------------------------
## Implementation

### Tokenising messages into simple words

In [5]:
# libraries
from typing import Set, NamedTuple
import re

In [2]:
def tokenise(text: str) -> Set[str]:
    text = text.lower()
    all_words = re.findall("[a-z0-9']+", text) # word extraction
    return set(all_words)

In [4]:
assert tokenise("Machine learning is about making a machine learn") == {'a', 'about', 'is', 'learn', 'learning', 'machine', 'making'}

In [6]:
class Message(NamedTuple):
    text: str
    is_spam: bool

- it is a `class` to keep track of `tokens`, `counts` and `labels` from the training data.
- non-spam emails will be categorised as `okay`